In [61]:
import pandas as pd

In [62]:
TXN_FILE = "transactions.csv"
REPAY_FILE = "repayments.csv"

In [63]:
txn_df = pd.read_csv(TXN_FILE)
repay_df = pd.read_csv(REPAY_FILE)

### Query 1

In [64]:
def calculate_total_outstanding():
    # Calculate the total principal to be paid for each loan (leaving out interest as per the query's description)
    principal_to_pay_series = txn_df['downPaymentAmount'] + (txn_df['tenure'] * txn_df['emiPrincipal'])
    principal_to_pay_df = pd.DataFrame({'transactionId' : txn_df['transactionId'], 'totalPrincipalToPay' : principal_to_pay_series})

    # Group repayments by transactionId and sum the repayments towards each individial loan
    total_repayments_df = repay_df.groupby('transactionId')['amount'].sum().reset_index()

    # Merge transactions with total repayments
    paid_unpaid_df = pd.merge(principal_to_pay_df, total_repayments_df, on='transactionId', how='left')
    paid_unpaid_df.rename(columns={'amount': 'totalPrincipalPaid'}, inplace=True)

    # Replace NaN with 0 for loans with no repayments (not the case in this dataset, but adding for future)
    paid_unpaid_df['totalPrincipalPaid'] = paid_unpaid_df['totalPrincipalPaid'].fillna(0)

    # Calculate outstanding amount to be paid
    paid_unpaid_df['outstandingAmount'] = (paid_unpaid_df['totalPrincipalToPay'] - paid_unpaid_df['totalPrincipalPaid'])

    # Select required columns
    final_result = pd.DataFrame(paid_unpaid_df[['transactionId', 'outstandingAmount']])
    # Convert paise to rupees
    final_result['outstandingAmount'] = final_result['outstandingAmount'] / 100
    return final_result


In [65]:
calculate_total_outstanding()

,transactionId,outstandingAmount
0,704663,0.00
1,1403240,0.00
2,1961345,0.00
3,1682956,0.00
4,2392323,2811.34
...,...,...
995,1076397,0.00
996,945211,0.00
997,541382,0.00
998,336716,0.00


### Query 2